In [1]:
import os
import pandas as pd

In [2]:
    df=pd.read_csv("C:/downloads/CCPerformance/vmCloud_data.csv", parse_dates=True)
    print(df.head())

                                  vm_id            timestamp  cpu_usage  \
0  c5215826-6237-4a33-9312-72c1df909881  2023-01-25 09:10:54  54.881350   
1  29690bc6-1f34-403b-b509-a1ecb1834fb8  2023-01-26 04:46:34  71.518937   
2  2e55abc3-5bad-46cb-b445-a577f5e9bf2a  2023-01-13 23:39:47        NaN   
3  e672e32f-c134-4fbc-992b-34eb63bef6bf  2023-02-09 11:45:49  54.488318   
4  f38b8b50-6926-4533-be4f-89ad11624071  2023-06-14 08:27:26  42.365480   

   memory_usage  network_traffic  power_consumption  \
0     78.950861       164.775973         287.808986   
1     29.901883              NaN         362.273569   
2     92.709195       203.674847         231.467903   
3     88.100960              NaN         195.639954   
4           NaN              NaN         359.451537   

   num_executed_instructions  execution_time  energy_efficiency task_type  \
0                     7527.0       69.345575           0.553589   network   
1                     5348.0       41.396040           0.349856 

In [3]:
import datetime

# Function to convert string to datetime
def convert(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    datetime_str = datetime.datetime.strptime(date_time, format)

    return datetime_str

In [17]:
import datetime
from datetime import timedelta
import numpy as np
import pickle

    
date_time = '2023-07-20 12:04:18'
# print(convert(date_time))
# print(type(convert(date_time)))
cur_time=convert(date_time)
#replacing with 2023 data for testing
# print('date_time: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))

df['timestamp']=pd.to_datetime(df['timestamp'])



vms=df["vm_id"].unique()
# print(f"Number of VMs: {len(vms)}")
# print(vms)
# print(type(vms))


# rec_df=df[df['timestamp']<=cur_time and df['timestamp']>=cur_time - timedelta(hours = 1)] #1 hoour for smoothing value calculation
mask=(df['timestamp']<=cur_time)  & (df['timestamp']>=cur_time - timedelta(days = 30)) #1 hoour for smoothing value calculation
rec_df=df.loc[mask]
# print(rec_df)
# print(rec_df.shape[0])
span=10

# Not enough log entries for prediction at VM level hence doing it at aggregate level
#aggregate level
df_vm=rec_df
df_vm.set_index("timestamp", inplace=True)
# print(df_vm.shape[0])
#mem
mem=df_vm["memory_usage"].to_frame()
mem_usage=mem["memory_usage"].ewm(span=span).mean()

#network_io
net=df_vm[df_vm['task_type']=='network']
net=net["network_traffic"].to_frame()
net_usage=net["network_traffic"].ewm(span=span).mean()

#local_io
io=df_vm[df_vm['task_type']=='io']
io=io["execution_time"].to_frame()
io_usage=io["execution_time"].ewm(span=span).mean()


#nfs_io
nfs=df_vm[df_vm['task_type']=='network']
nfs=nfs["execution_time"].to_frame()
nfs_usage=nfs["execution_time"].ewm(span=span).mean()

# print(type(mem_usage))

ins=np.array([mem_usage.iloc[0], net_usage.iloc[0], io_usage.iloc[0], nfs_usage.iloc[0]])
test=ins

clf = pickle.load(open("../models/stacking", 'rb')) 
final_model = pickle.load(open("../models/voting", 'rb')) 

# print(test)
# print(test.shape)

remediation=0
if clf.predict([test]) == 1: # and clf.predict_proba() >=0.5:
    print(f"VM  is going to fail soon **\n")
    remediation=1
    
elif  final_model.predict([test]) == 1:# and final_model.predict_proba() >=0.5:
    print(f"VM  is going to fail soon **\n")
    remediation=1
else:
    print(f"VM  is doing good **\n")

if remediaton:
    print("remediation required:\n")
    # prob0=clf.predict_proba(test)
    cause=-1
    causes=list("mem_usage", "net_usage", "io_usage", "nfs_usage")
    for i, c  in enumerate(causes):
        print(i)
        prob[i]=1
    for i, c  in enumerate(causes):
        ins[i]=0
        if clf.predict(ins)==0:
            cause=i
            break
        else:
            prob[i]=clf.predict_proba(ins)

        ins=test

    if cause ==-1:
        cause = np.where(prob == np.min(prob))[0]


    match cause:
        case '0': 
            print("Increase memory\n")
            
        case '1': 
            print("Increase network bandwidth\n")
            
        case '2': 
            print("Use VM of higher configuration\n")
            
        case '3': 
            print("Use better design/architecture to reduce nfs calls\n")
            

                               
        

        
        
        
            
        
        
        
        
    
    
                      
# #VM level
# for i, v in enumerate(vms):
#     df_vm=rec_df[rec_df['vm_id']==v]
#     print(df_vm.shape[0])    
#     if df_vm.shape[0] <span:
#         continue

#     df_vm.set_index("timestamp", inplace=True)
#     print(df_vm.shape[0])
#     #mem
#     mem=df_vm["memory_usage"].to_frame()
#     mem_usage=mem["memory_usage"].ewm(span=span).mean()

#     #network_io
#     net=df_vm[df_vm['task_type']=='network']
#     net=net["network_traffic"].to_frame()
#     net_usage=net["network_traffic"].ewm(span=span).mean()

#     #local_io
#     io=df_vm[df_vm['task_type']=='io']
#     io=io["execution_time"].to_frame()
#     io_usage=io["execution_time"].ewm(span=span).mean()


#     #nfs_io
#     nfs=df_vm[df_vm['task_type']=='network']
#     nfs=nfs["execution_time"].to_frame()
#     nfs_usage=nfs["execution_time"].ewm(span=span).mean()

#     ins=[mem_usage.iloc[0], net_usage.iloc[0], io_usage.iloc[0], nfs_usage.iloc[0]]

#     test=pd.DataFrame(ins, columns=['memory_GB',  'network_log10_MBps',  'local_IO_log10_MBps',  'NFS_IO_log10_MBps'])
#     print(test)

#     if clf.predict(test) == 1: # and clf.predict_proba() >=0.5:
#         print(f"VM {v} is going to fail soon **\n")
#         continue
        
#     if final_model.predict(test) == 1:# and final_model.predict_proba() >=0.5:
#         print(f"VM {v} is going to fail soon **\n")
#         continue
    
#     print(f"VM {v} is doing good **\n")
    
                
        
    
    

    

VM  is doing good **



C:\Users\fahmi\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\fahmi\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\fahmi\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(
C:\Users\fahmi\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
C:\Users\fahmi\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feat

NameError: name 'remediaton' is not defined